In [14]:
import numpy as np
from glob import glob
import os
import random
import cv2
from PIL import Image

from scipy import ndimage
from IPython.display import display, display_markdown

def motion_mag_downsample(mag, size, input_size):
    block_size = input_size // size
    mask = np.zeros((size,size))
    for i in range(size):
        for j in range(size):
            x_start = i * block_size
            x_end = x_start + block_size
            y_start = j * block_size
            y_end = y_start + block_size

            tmp_block = mag[x_start:x_end, y_start:y_end]

            block_mean = np.mean(tmp_block)
            mask[i, j] = block_mean
    return mask


def compute_motion_boudary(flow_clip):
    mx = np.array([[-1, 0, 1], [-1, 0, 1], [-1, 0, 1]])
    my = np.array([[-1, -1, -1], [0, 0, 0], [1, 1, 1]])
    dx_all = []
    dy_all = []
    mb_x = 0
    mb_y = 0

    for flow_img in flow_clip:
        d_x = ndimage.convolve(flow_img, mx)
        d_y = ndimage.convolve(flow_img, my)

        dx_all.append(d_x)
        dy_all.append(d_y)

        mb_x += d_x
        mb_y += d_y

    dx_all = np.array(dx_all)
    dy_all = np.array(dy_all)

    return dx_all, dy_all, mb_x, mb_y

file = '/home/lr/dataset/YouTube-VOS/2019/train_all_frames/Flows/c3be369fdb'
flows = glob(os.path.join(file, '*.jpg'))

flow = flows[random.randint(0, len(flows) - 1)]

flow = cv2.imread(flow)
flow = cv2.resize(flow, (256,256))

u, v = [flow[:,:,0].astype(np.float32)], [flow[:,:,1].astype(np.float32)]

_, _ , mb_x_u, mb_y_u = compute_motion_boudary(u)
mag_u, ang = cv2.cartToPolar(mb_x_u, mb_y_u, angleInDegrees=True)
mag_down_u = motion_mag_downsample(mag_u, 8, 256)

_, _ , mb_x_v, mb_y_v = compute_motion_boudary(v)
mag_v, ang = cv2.cartToPolar(mb_x_v, mb_y_v, angleInDegrees=True)
mag_down_v = motion_mag_downsample(mag_v, 8, 256)

mag_down_u = (mag_down_u-mag_down_u.min()) * 255 / (mag_down_u.max() - mag_down_u.min()).astype(np.uint8)
mag_down_v = (mag_down_v-mag_down_v.min()) * 255 / (mag_down_v.max() - mag_down_v.min()).astype(np.uint8)

mag = (mag_down_u + mag_down_v) // 2
mag = Image.fromarray(mag).convert('L')


display(mag)


